In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [2]:
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116 (from 1)
Receiving objects: 100% (134/134), 31.28 MiB | 27.87 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/bleurt
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=12f0ba7aa08d33e42849d782e6ca2bc2d6a0090e33aaf7575ae32a9ba8a4313f
  Stored in directory: /tmp/pip-ephem-wheel-cache-d0ld7f1o/wheels/49/ab/73/9318ab38d4cd1c732bcea8335d3f8d7c0316c8d07b9084fa85
Successfully built BLEURT


In [3]:
!pip install -q evaluate rouge_score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.2 MB/s eta 0:00:00


In [4]:
import json
import time
import os
import pandas as pd
import numpy as np
import re
from copy import deepcopy

In [5]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# !python -m unittest bleurt.score_test
# !python -m unittest bleurt.score_not_eager_test
# !python -m unittest bleurt.finetune_test
# !python -m unittest bleurt.score_files_test

In [6]:
from bleurt import score
from evaluate import load as load_metric

In [7]:
rouge = load_metric("rouge")
bertscore = load_metric("bertscore")
bleurt = load_metric("bleurt", module_type="metric")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Load generations

In [9]:
fine_tune_path = '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM'
model_name = 'pegasus' # ADJUST
sub_model_name = 'pegasus_xsum' # ADJUST
checkpoints_dir = f"{fine_tune_path}/{model_name}/{sub_model_name}/lora_1" # ADJUST
checkpoints_path = f"{checkpoints_dir}/checkpoints"
final_checkpoints_path = f"{checkpoints_dir}/final_checkpoints"
eval_path = f"{checkpoints_dir}/eval_{sub_model_name}.csv"

In [10]:
eval_path

'/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/pegasus/pegasus_xsum/lora_1/eval_pegasus_xsum.csv'

In [13]:
test_soap_50_df = pd.read_csv(f"{checkpoints_dir}/test_soap_50.csv")
test_soap_50_100_df = pd.read_csv(f"{checkpoints_dir}/test_soap_50_100.csv")
# train_soap_50_df = pd.read_csv(f"{checkpoints_dir}/train_soap_50.csv")
mts_50_df = pd.read_csv(f"{checkpoints_dir}/mts_50.csv")
aci_50_df = pd.read_csv(f"{checkpoints_dir}/aci_50.csv")

# Evaluate on MTS-Dialog

## Testing

In [30]:
def evaluate_clinical_summary(generated_text: str, reference_text: str) -> dict:
    predictions = [generated_text]
    references = [reference_text]

    # ROUGE
    rouge_scores = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # BERTScore
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="en")
    bert_precision = sum(bert_scores['precision']) / len(bert_scores['precision'])
    bert_recall = sum(bert_scores['recall']) / len(bert_scores['recall'])
    bert_f1 = sum(bert_scores['f1']) / len(bert_scores['f1'])

    # BLEURT
    bleurt_score = bleurt.compute(predictions=predictions, references=references)['scores'][0]

    # # QuestEval
    # questeval_score = questeval.compute(hypothesis=generated_text, reference=reference_text, do_weighing=False)['questeval_score']

    return {
        "ROUGE-1": round(rouge_scores['rouge1'], 4),
        "ROUGE-2": round(rouge_scores['rouge2'], 4),
        "ROUGE-L": round(rouge_scores['rougeL'], 4),
        "ROUGE-LSum": round(rouge_scores['rougeLsum'], 4),
        "BERTScore-P": round(bert_precision, 4),
        "BERTScore-R": round(bert_recall, 4),
        "BERTScore-F1": round(bert_f1, 4),
        "BLEURT": round(bleurt_score, 4),
        # "QuestEval": round(questeval_score, 4)
    }

In [17]:
test_soap_50_df.head()

,dialogue,reference_note,fine_tuned_model_output
0,"Doctor: Hello, can you please tell me about yo...","S: The patient, a flooring installer with no s...","S: The patient, a flooring installer, reports ..."
1,"Doctor: Hello, I understand that you're a 7-ye...",S: The patient is a 7-year-old boy with congen...,S: The patient is a 7-year-old boy with congen...
2,"Doctor: Hello, we've received your results fro...",S: The patient reported undergoing an ultrasou...,S: The patient reports a single thyroid nodule...
3,"Doctor: Hello, can you tell me what brought yo...","S: The patient reports a progressive headache,...","S: The patient reports a progressive headache,..."
4,"Doctor: Hello, I understand that you have been...","S: The patient, a post-liver transplant recipi...","S: The patient, a patient with a history of ca..."


In [18]:
evaluate_clinical_summary("This is a test", "This is a test")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'ROUGE-1': np.float64(1.0),
 'ROUGE-2': np.float64(1.0),
 'ROUGE-L': np.float64(1.0),
 'ROUGE-LSum': np.float64(1.0),
 'BERTScore-P': 1.0,
 'BERTScore-R': 1.0,
 'BERTScore-F1': 1.0,
 'BLEURT': 1.0118}

In [21]:
test_soap_50_df.iloc[0]["reference_note"]

"S: The patient, a flooring installer with no significant past medical history, presents with painless blurry vision in the right eye for one week, intermittent fevers, headaches, body aches, and a nonpruritic maculopapular rash on the lower legs for six months. The patient also reports an isolated episode of left knee and testicular swelling in the past but denies any neck stiffness, nausea, vomiting, Raynaud's phenomenon, oral ulcerations, chest pain, shortness of breath, abdominal pain, or photosensitivity. No history of exposure to toxic substances or habits related to smoking, drinking, or illicit drug use.\nO: Vital signs are normal. Physical examination reveals bilateral papilledema, greater optic nerve erythema in the right eye, a right inferior nasal quadrant visual field defect, and a right afferent pupillary defect. Muscle strength, reflexes, and sensation to light touch, pinprick, vibration, and proprioception are intact. A maculopapular rash is noted on bilateral lower ext

In [22]:
test_soap_50_df.iloc[0]["fine_tuned_model_output"]

'S: The patient, a flooring installer, reports painless blurry vision in the right eye for a week, intermittent fevers, headache, body aches, and a nonpruritic maculopapular rash on the lower legs for the past 6 months. He denies any past medical history, including an isolated episode of left knee swelling and testicular swelling in the past. He also denies any toxic substances, smoking, drinking, or illicit drug use. O: Vital signs were normal. Physical examination revealed bilateral papilledema and optic nerve in the right eye, bilateral inferior nasal quadrant visual field defect, and right a pupillary defect. Muscle strength and reflexes were normal. Muscle strength and reflexes were intact. Laboratory findings included microcytic sedimentation with a magnification of 11.6 m/dL, hematocrit of 35.3%, and mean corpuscular volume of 76.9 fL. ESR was elevated at 33 mm/hr, and C-reactive protein (CRP) was elevated at 13.3 mg/L. A: The primary diagnosis is a suspected inflammatory or inf

In [31]:
evaluate_clinical_summary(test_soap_50_df.iloc[0]["reference_note"], test_soap_50_df.iloc[0]["fine_tuned_model_output"])

{'ROUGE-1': np.float64(0.6476),
 'ROUGE-2': np.float64(0.3486),
 'ROUGE-L': np.float64(0.4755),
 'ROUGE-LSum': np.float64(0.4789),
 'BERTScore-P': 0.8789,
 'BERTScore-R': 0.9152,
 'BERTScore-F1': 0.8967,
 'BLEURT': 0.42}

In [24]:
evaluate_clinical_summary(test_soap_50_df.iloc[1]["reference_note"], test_soap_50_df.iloc[1]["fine_tuned_model_output"])

{'ROUGE-1': np.float64(0.5477),
 'ROUGE-2': np.float64(0.2929),
 'ROUGE-L': np.float64(0.4673),
 'ROUGE-LSum': np.float64(0.4623),
 'BERTScore-P': 0.8998,
 'BERTScore-R': 0.8747,
 'BERTScore-F1': 0.887,
 'BLEURT': 0.1616}

## Sequential

In [ ]:
def evaluate_sequentially(df):
    results = []

    for idx, sample in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
        dialogue = sample["dialogue"]
        reference_summary = sample["reference summary"]
        generated_summary = sample["generated summary"]

        metrics = evaluate_clinical_summary(generated_summary, reference_summary)

        results.append({
            "dialogue": dialogue,
            "reference_summary": reference_summary,
            "generated_summary": generated_summary,
            **metrics
        })

    return pd.DataFrame(results)

In [ ]:
eval_test_soap_50_df = evaluate_sequentially(test_soap_50_df)
eval_test_soap_50_df.to_csv(f"{checkpoints_dir}/eval_test_soap_50.csv", index=False)

## Parallel

In [27]:
# Parallel
def evaluate_in_batch(df, batch_size=16):
    results = []

    for start in tqdm(range(0, len(df), batch_size), desc="Evaluating in batches"):
        batch = df.iloc[start:start+batch_size]
        preds = batch["fine_tuned_model_output"].tolist()
        refs = batch["reference_note"].tolist()
        diags = batch["dialogue"].tolist()

        # Compute metrics
        # rouge_scores = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
        bert_scores = bertscore.compute(predictions=preds, references=refs, lang="en", device="cuda")
        bleurt_scores = bleurt.compute(predictions=preds, references=refs)

        for i in range(len(preds)):
          # ROUGE must be computed per example
            rouge_score = rouge.compute(predictions=[preds[i]], references=[refs[i]], use_stemmer=True)

            results.append({
                "dialogue": diags[i],
                "reference_summary": refs[i],
                "generated_summary": preds[i],
                "ROUGE-1": round(rouge_score['rouge1'], 4),
                "ROUGE-2": round(rouge_score['rouge2'], 4),
                "ROUGE-L": round(rouge_score['rougeL'], 4),
                "ROUGE-LSum": round(rouge_score['rougeLsum'], 4),
                "BERTScore-P": round(bert_scores['precision'][i], 4),
                "BERTScore-R": round(bert_scores['recall'][i], 4),
                "BERTScore-F1": round(bert_scores['f1'][i], 4),
                "BLEURT": round(bleurt_scores['scores'][i], 4),
            })

    return pd.DataFrame(results)

In [29]:
test_df = evaluate_in_batch(test_soap_50_df[:4], batch_size=2)
test_df

Evaluating in batches: 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


,dialogue,reference_summary,generated_summary,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-LSum,BERTScore-P,BERTScore-R,BERTScore-F1,BLEURT
0,"Doctor: Hello, can you please tell me about yo...","S: The patient, a flooring installer with no s...","S: The patient, a flooring installer, reports ...",0.6476,0.3486,0.4755,0.5126,0.9152,0.8789,0.8967,0.3707
1,"Doctor: Hello, I understand that you're a 7-ye...",S: The patient is a 7-year-old boy with congen...,S: The patient is a 7-year-old boy with congen...,0.5477,0.2929,0.4673,0.4774,0.8747,0.8998,0.8870,0.2163
2,"Doctor: Hello, we've received your results fro...",S: The patient reported undergoing an ultrasou...,S: The patient reports a single thyroid nodule...,0.5507,0.2578,0.3644,0.4182,0.9006,0.8829,0.8917,-0.3796
3,"Doctor: Hello, can you tell me what brought yo...","S: The patient reports a progressive headache,...","S: The patient reports a progressive headache,...",0.6596,0.3737,0.4693,0.5032,0.9043,0.9130,0.9086,0.3830


In [32]:
eval_test_soap_50_df = evaluate_in_batch(test_soap_50_df, batch_size=16)
eval_test_soap_50_df.to_csv(f"{checkpoints_dir}/eval_test_soap_50.csv", index=False)

Evaluating in batches: 100%|██████████| 4/4 [00:17<00:00,  4.31s/it]


In [33]:
eval_test_soap_50_df

,dialogue,reference_summary,generated_summary,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-LSum,BERTScore-P,BERTScore-R,BERTScore-F1,BLEURT
0,"Doctor: Hello, can you please tell me about yo...","S: The patient, a flooring installer with no s...","S: The patient, a flooring installer, reports ...",0.6476,0.3486,0.4755,0.5126,0.9152,0.8789,0.8967,0.3707
1,"Doctor: Hello, I understand that you're a 7-ye...",S: The patient is a 7-year-old boy with congen...,S: The patient is a 7-year-old boy with congen...,0.5477,0.2929,0.4673,0.4774,0.8747,0.8998,0.8870,0.2163
2,"Doctor: Hello, we've received your results fro...",S: The patient reported undergoing an ultrasou...,S: The patient reports a single thyroid nodule...,0.5507,0.2578,0.3644,0.4182,0.9006,0.8829,0.8917,-0.3796
3,"Doctor: Hello, can you tell me what brought yo...","S: The patient reports a progressive headache,...","S: The patient reports a progressive headache,...",0.6596,0.3737,0.4693,0.5032,0.9043,0.9130,0.9086,0.3830
4,"Doctor: Hello, I understand that you have been...","S: The patient, a post-liver transplant recipi...","S: The patient, a patient with a history of ca...",0.4909,0.2384,0.2978,0.3622,0.8894,0.8743,0.8818,-0.1023
5,"Doctor: Hello, how can I help you today?\nPati...","S: The patient, a non-smoker and engineer, rep...","S: The patient, a non-smoker, reports experien...",0.5855,0.3156,0.3969,0.4519,0.9000,0.8841,0.8919,-0.0618
6,"Doctor: Hi there, I see you've presented with ...",S: The patient presented with body weight loss...,"S: The patient, with hypertension and diabetes...",0.5535,0.2947,0.3396,0.4235,0.8813,0.8812,0.8812,-0.1057
7,"Doctor: Hello, I heard you have a 6-month hist...",S: The patient reports a 6-month history of sw...,S: The patient reports a 6-month history of sw...,0.7171,0.4902,0.5702,0.5875,0.9342,0.9117,0.9228,0.1387
8,"Doctor: Hi there, can you please tell me about...","S: The patient, a male with a history of prost...","S: The patient, a prostate cancer patient, rep...",0.6679,0.3672,0.4618,0.4867,0.9285,0.9120,0.9202,0.0137
9,"Doctor: Hello, how can I help you today?\nPati...",S: The patient is a 76-year-old woman presenti...,"S: The patient, a 76-year-old woman, reports a...",0.6429,0.3333,0.4286,0.4893,0.8992,0.8985,0.8989,0.2592


In [34]:
eval_test_soap_50_100_df = evaluate_in_batch(test_soap_50_100_df, batch_size=16)
eval_test_soap_50_100_df.to_csv(f"{checkpoints_dir}/eval_test_soap_50_100.csv", index=False)

Evaluating in batches: 100%|██████████| 4/4 [00:18<00:00,  4.68s/it]


In [36]:
eval_test_soap_50_100_df.head(3)

,dialogue,reference_summary,generated_summary,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-LSum,BERTScore-P,BERTScore-R,BERTScore-F1,BLEURT
0,"Doctor: Hello, what brings you in today?\nPati...",S: The patient reports severe pain in the lowe...,S: The patient reports a golf ball incident in...,0.6455,0.3777,0.3651,0.4550,0.9211,0.9121,0.9166,-0.0994
1,"Doctor: Hello, how can I help you today?\nPati...",S: The patient is a 65-year-old male presentin...,S: The patient is a 65-year-old male with a hi...,0.7074,0.4144,0.5095,0.5137,0.9207,0.9159,0.9183,-0.1079
2,"Doctor: Hello, I understand that you have a hi...",S: Patient reports a one-week history of fever...,"S: The patient, a breast abscess patient with ...",0.6233,0.3604,0.4970,0.5128,0.9053,0.9087,0.9070,-0.3219


In [37]:
eval_mts_50_df = evaluate_in_batch(mts_50_df, batch_size=16)
eval_mts_50_df.to_csv(f"{checkpoints_dir}/eval_mts_50.csv", index=False)

Evaluating in batches: 100%|██████████| 4/4 [00:14<00:00,  3.70s/it]


In [38]:
eval_mts_50_df.head(3)

,dialogue,reference_summary,generated_summary,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-LSum,BERTScore-P,BERTScore-R,BERTScore-F1,BLEURT
0,Doctor: What brings you back into the clinic t...,The patient is a 76-year-old white female who ...,"S: Patient, a seventy six-year-old white femal...",0.3951,0.2324,0.3045,0.3045,0.8751,0.8951,0.8850,-0.3855
1,Doctor: How're you feeling today? \nPatient: ...,The patient is a 25-year-old right-handed Cauc...,"S: The patient, a twenty-five-year-old female,...",0.3343,0.0917,0.1945,0.1945,0.8262,0.8540,0.8399,-0.4792
2,"Doctor: Hello, miss. What is the reason for yo...","This is a 22-year-old female, who presented to...","S: The patient, a twenty two year old female, ...",0.2312,0.1218,0.1809,0.1809,0.8423,0.8836,0.8625,-0.5436


In [39]:
eval_aci_50_df = evaluate_in_batch(aci_50_df, batch_size=16)
eval_aci_50_df.to_csv(f"{checkpoints_dir}/eval_aci_50.csv", index=False)

Evaluating in batches: 100%|██████████| 4/4 [00:19<00:00,  4.97s/it]


In [40]:
eval_aci_50_df.head(3)

,dialogue,reference_summary,generated_summary,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-LSum,BERTScore-P,BERTScore-R,BERTScore-F1,BLEURT
0,"[doctor] hi , martha . how are you ?\n[patient...",CHIEF COMPLAINT\n\nAnnual exam.\n\nHISTORY OF ...,S: Martha is a 50-year-old female with a histo...,0.3659,0.1407,0.2256,0.3258,0.8456,0.8083,0.8266,-0.6870
1,"[doctor] hi , andrew , how are you ?\n[patient...",CHIEF COMPLAINT\n\nJoint pain.\n\nHISTORY OF P...,"S: The patient, a 62-year-old male with a hist...",0.3539,0.1128,0.1948,0.3347,0.8708,0.8212,0.8453,-0.4522
2,"[doctor] hi , john . how are you ?\n[patient] ...",CHIEF COMPLAINT\n\nBack pain.\n\nHISTORY OF PR...,"S: John, a 61-year-old male with a history of ...",0.3978,0.1785,0.2142,0.3421,0.8612,0.8151,0.8375,-0.2546
